# Capstone Project

# Libraries

In [1]:
# Import Library 

import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from pprint import pprint
import numpy as np
from numpy import array
import pandas as pd 
import csv
import pickle
from IPython.display import display
import pickle
from collections import Counter
pd.set_option('display.max_columns', 100)
from IPython.core import display as ICD


# Define Text Normalization Function

In [2]:
def restore_windows_1252_characters(restore_string):
    """
    Replace C1 control characters in the Unicode string s by the
    characters at the corresponding code points in Windows-1252,
    where possible.
    """

def to_windows_1252(match):
    try:
        return bytes([ord(match.group(0))]).decode('windows-1252')
    except UnicodeDecodeError:
        # No character at the corresponding code point: remove it.
        return ''

    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Grab the Document Content

In [3]:
def grab_doc_content( brand, CIK ):
    
    company = {}
    
    company['auto'] = {}
    auto = {brand : CIK
           }
    company['auto'] = auto

    value = list(auto.copy().values())
    
    return value

In [4]:
GDC = grab_doc_content('Amazon', '0001018724')

In [5]:
master_filings_dict = {}
    
file_code = {}
    
file_text = {}

Accession_Number_URL = {}

In [6]:
# URL Directory For CIK    
dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
dir_url_list = [dir_url.format(x) for x in GDC]

#print('Directory URL: {}'.format(dir_url_list))
doc_url_list = [] 

# FOR-loop yielding Accession Numbers from CIK/URL Directory.
for CIK_num in GDC:

    doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
    doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
    doc_url_list.append(doc_url_new)


In [7]:
def url_maker(dir_url):
    dir_url_list = [dir_url.format(x) for x in GDC]

    #print('Directory URL: {}'.format(dir_url_list))
    doc_url_list = [] 

    # FOR-loop yielding Accession Numbers from CIK/URL Directory.
    for CIK_num in GDC:

        doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
        doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
        
        doc_url_list.append(doc_url_new)
        
        url_lists = zip( dir_url_list , doc_url_list )
        
    return url_lists

In [8]:
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
for dir_url, doc_url in url_maker(url): 
    print(1, dir_url)
    print(2, doc_url)

1 https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001018724&type=10-&dateb=&owner=include&count=100
2 https://www.sec.gov/Archives/edgar/data/0001018724/{xx}/{yy}.txt


In [9]:
Accession_Number_URL = {}
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

for dir_url, doc_url in url_maker(url):
    
    response = requests.get(dir_url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text(strip=True)

    cleaned_text = re.findall('Acc-no: \d+-\d+-\d+' , text)

    accession_number = [n.replace('Acc-no: ', '') for n in cleaned_text]
    accessionnumber = [num.replace('-', '') for num in accession_number]

    accession_numbers = zip(accessionnumber, accession_number)
    
    cikk = [ cikk_.replace('CIK=', '') for cikk_ in re.findall('CIK=\d+', dir_url) ][0]
    CIKK = {cikk : accessionnumber}
    
    for (a,b) in accession_numbers: 
        master_filings_dict[b] = {}
        master_filings_dict[b]['sec_header_content'] = {}
        master_filings_dict[b]['filing_documents'] = None

        doc_url_single = doc_url.format(xx = a, yy = b)

        file_url_list = []

        

        file_url_list.append( doc_url_single )
        
        Accession_Number_URL.update({ b : file_url_list})

In [10]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    if AN == '0001445305-13-002495':
        break
    else:
        A_N.append(AN)
print(A_N, '\n')
print(len(A_N))

['0001018724-20-000010', '0001018724-20-000004', '0001018724-19-000089', '0001018724-19-000071', '0001018724-19-000043', '0001018724-19-000004', '0001018724-18-000159', '0001018724-18-000108', '0001018724-18-000072', '0001018724-18-000005', '0001018724-17-000135', '0001018724-17-000100', '0001018724-17-000051', '0001018724-17-000011', '0001018724-16-000324', '0001018724-16-000286', '0001018724-16-000227', '0001018724-16-000172', '0001018724-15-000126', '0001018724-15-000087', '0001018724-15-000038', '0001018724-15-000006', '0001018724-14-000055', '0001018724-14-000025', '0001018724-14-000014', '0001018724-14-000006'] 

26


## Save each Filing.

In [11]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0000320193-20-000010':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()

        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('hr')

        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]

        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))
    


--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 45 page(s) found.
There was 45 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001018724-20-000010 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 75 page(s) found.
There was 75 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001018724-20-000004 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 44 page(s) found.
There was 44 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001018724-19-000089 were parsed and stored.
-

--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 72 page(s) found.
There was 72 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001018724-14-000006 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 40 page(s) found.
There was 40 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001445305-13-002495 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 37 page(s) found.
There was 37 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001193125-13-303204 were parsed and stored.
-

--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 84 page(s) found.
There was 84 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001193125-08-024707 were parsed and stored.


AttributeError: 'NavigableString' object has no attribute 'get_text'

In [ ]:
string_test = master_filings_dict['0000320193-20-000052']['filing_documents']['10-Q']['pages_code']


## Anchor Check Function



In [16]:
# Variations of HTML anchors.
CBS_A = 'CONDENSED CONSOLIDATED BALANCE SHEETS'
CBS_B = 'CONSOLIDATED BALANCE SHEETS'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) or (CBS_B in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


## Cleaning Columns Function


In [17]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [18]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

In [19]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

In [20]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        try:
            ele= element.encode('cp1252').decode("utf-8", "ignore")
                
        except:
            ele= element.encode('latin1').decode("utf-8", "ignore")
            
        if ele =='':
            continue
            
        ele_norm = unicodedata.normalize('NFKD', ele)
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif ele_norm not in ['\n$', '\n\xa0', '\n', '\xa0']:
            
            n_element = ele_norm.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)
            
            result = [int(clean_ele) * m for clean_ele in res]

            column_list.extend(result)

    return column_list
    

## Cleaning Tables Function

In [161]:
# New and imporved.
def Quarter( column , k):
    
#     print('column used for quarter', column)
    
    multi_list = [element.split() for element in column if element.split() != []]
#     print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
#     print('new_column used for year', new_column)
    
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        try:
            month = re.findall(r'\w{}+'.format(list(month_dict.keys())), letter.replace(' ',  '') )[0]
            
            Q_num = month_dict[month] 
    #         print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
    print('Q' , Q) 
        
    if k == '10-K':
        Q_final =['Quarter'] + Qm
        Q_final.insert(1, 0)
        
        
    for letter in new_column:   
        
        #print( letter.split(',') )
        
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
#     print(f'Y - {Y}')   # Y - [[], [], [['2015']], [['2016']]]
    
    Y = sorted([num for sublist in Y for item in sublist for num in item])
    print(f'Y - {Y}')
    
#     if k == '10-K' and int(Y[0]) < 2017:
#         Y = [str(int(Y[0]) + 1)]
#         print(f'Changed Y into {Y}')

    
        
    if k == '10-Q' and int(Y[0]) > 2015:
        Q_final =['Quarter'] + sorted(Q, reverse=True)
            
    else:
        Q_final =['Quarter'] + sorted(Q)
    
    
    if k == '10-Q' and int(Y[0]) < 2016:
        Y = sorted(Y, reverse=True)
        Y.insert(0, 'Year')
        
    else:    
        Y.insert(0, 'Year')
    
    #Y.insert(1, 0)

    
    return Q_final, Y
    print( '\n', 'Q_final', Q_final, 'Y', Y)

In [185]:
Checklist = {}

for k, v in QorK_10Q_2.items():
    value_list = []
    for i, vv in enumerate(v):
        year = int(vv.split(sep='-')[1])
        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

#         try:
        # Extract Table from html code and wrap it as a list of df.
        table_df = table(pages, k)[0]  
        
        if year > 16:
            table_df = table_df.drop([1]).reset_index(drop=True)
            
        else: 
            table_df = table_df.drop([2]).reset_index(drop=True)
        
        ICD.display(table_df)

        with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
            pickle.dump(table_df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
#         except: 
#             value_list.append(vv)
#             print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
#     del value_list

Q ['Q1', 'Q4']
Y - ['2012', '2013']


0 Quarter  Year   NaN ASSETS Current assets        Cash Marketable securities  \
0      Q1  2013  None   None           None  4481000000            3414000000   

0   Inventory Accounts Receivable Deferred tax assets   NaN  \
0  5395000000          2516000000           507000000  None   

0 Total current assets Property and Equipment Deferred tax assets    Goodwill  \
0          16313000000             7674000000           123000000  2535000000   

0 Other assets   NaN Total assets   NaN Shareholder Equity  \
0   1732000000  None  28377000000  None               None   

0 Current liabilities Accounts payable Accrued expenses and other   NaN  \
0                None       8916000000                 5416000000  None   

0 Total current liabilities Long term debt Other long term liabilities  \
0               14332000000     3040000000                  2573000000   

0 Commitments and contingencies Shareholder Equity Preferred Stock  \
0                          None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  479 and 478  \
0                 None                       None   

0 Outstanding shares  455 and 454 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                 8585000000   

0 Accumulated other comprehensive loss Retained Earnings   NaN  \
0                            319000000        1998000000  None   

0 Shareholder Equity   NaN Total liabilities and equity   NaN  
0         8432000000  None                  28377000000  None

All the pages from 10-Q document 0001193125-13-174667 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q3', 'Q4']
Y - ['2011', '2012']


0 Quarter  Year ASSETS Current assets        Cash Marketable securities  \
0      Q3  2012   None           None  2980000000            2268000000   

0   Inventory Accounts Receivable Deferred tax assets   NaN  \
0  5065000000          2392000000           413000000  None   

0 Total current assets Fixed assets Deferred tax assets    Goodwill  \
0          13118000000   5662000000            38000000  2540000000   

0 Other assets   NaN Total assets   NaN Shareholder Equity  \
0   1476000000  None  22834000000  None               None   

0 Current liabilities Accounts payable Accrued expenses and other   NaN  \
0                None       8369000000                 4236000000  None   

0 Total current liabilities Long term liabilities  \
0               12605000000            2676000000   

0 Commitments and contingencies Shareholder Equity Preferred Stock  \
0                          None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  476 and 473  \
0                 None                       None   

0 Outstanding shares  453 and 455 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                 7863000000   

0 Accumulated other comprehensive loss Retained Earnings   NaN  \
0                            296000000        1818000000  None   

0 Shareholder Equity   NaN Total liabilities and equity   NaN  
0         7553000000  None                  22834000000  None

All the pages from 10-Q document 0001193125-12-436012 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q2', 'Q4']
Y - ['2011', '2012']


...

All the pages from 10-Q document 0001193125-08-214483 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q2', 'Q4']
Y - ['2007', '2008']


0 Quarter  Year ASSETS Current assets        Cash Marketable securities  \
0      Q2  2008   None           None  1548000000             832000000   

0   Inventory Accounts Receivable Deferred tax assets   NaN  \
0  1107000000           586000000           163000000  None   

0 Total current assets Fixed assets Deferred tax assets   Goodwill  \
0           4236000000    651000000           284000000  400000000   

0 Other assets   NaN Total assets   NaN Shareholder Equity  \
0    751000000  None   6322000000  None               None   

0 Current liabilities Accounts payable Accrued expenses and other  \
0                None       1963000000                  812000000   

0 Current portion of long term debt   NaN Total current liabilities  \
0                         441000000  None                3216000000   

0 Long term debt Other long term liabilities Commitments and contingencies  \
0      433000000                   443000000                          None   

0 Shareholder Equity Preferred Stock Authorized shares  500  \
0               None            None                   None   

0 Issued and outstanding shares  none Common Stock Authorized shares  5  \
0                                None         None                 None   

0 Issued shares  440 and 431 Outstanding shares  426 and 416 Treasury stock  \
0                       None                         4000000      500000000   

0 Additional paid in capital Accumulated other comprehensive income  \
0                 3794000000                                6000000   

0 Accumulated deficit   NaN Shareholder Equity   NaN  \
0          1074000000  None         2230000000  None   

0 Total liabilities and equity   NaN  
0                   6322000000  None

All the pages from 10-Q document 0001193125-08-157451 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q1', 'Q4']
Y - ['2007', '2008']


0 Quarter  Year ASSETS Current assets        Cash Marketable securities  \
0      Q1  2008   None           None  1496000000             655000000   

0   Inventory Accounts Receivable Deferred tax assets   NaN  \
0  1077000000           581000000           156000000  None   

0 Total current assets Fixed assets Deferred tax assets   Goodwill  \
0           3965000000    594000000           280000000  392000000   

0 Other assets   NaN Total assets   NaN Shareholder Equity  \
0    652000000  None   5883000000  None               None   

0 Current liabilities Accounts payable Accrued expenses and other  \
0                None       1864000000                  781000000   

0 Current portion of long term debt   NaN Total current liabilities  \
0                         906000000  None                3551000000   

0 Long term debt Other long term liabilities Commitments and contingencies  \
0      467000000                   395000000                          None   

0 Shareholder Equity Preferred Stock Authorized shares  500  \
0               None            None                   None   

0 Issued and outstanding shares  none Common Stock Authorized shares  5  \
0                                None         None                 None   

0 Issued shares  432 and 431 Outstanding shares  417 and 416 Treasury stock  \
0                       None                         4000000      500000000   

0 Additional paid in capital Accumulated other comprehensive income  \
0                 3191000000                                7000000   

0 Accumulated deficit   NaN Shareholder Equity   NaN  \
0          1232000000  None         1470000000  None   

0 Total liabilities and equity   NaN  
0                   5883000000  None

All the pages from 10-Q document 0001193125-08-089548 have been tableized and saved.
--------------------------------------------------------------------------------


In [184]:
def table_extractor( page, multiplier, k):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    # then get all the rows in the table.
    table_rows = soup.find_all('tr')
    
    if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
        del table_rows[0]
    
    single_table = []
    Date_column = []
    
    # Loop through each column, adding in three structures to single_table.
    for tr_post, tr in enumerate(table_rows):
        
        td = tr.find_all('td')

        column = [tr.text for tr in td]
#         print( tr_post, len(column), column)
        # 10-Q 
        if k == '10-Q':
            
            
            if tr_post == 1 or tr_post == 2:
                Date_column.extend(column)
            
            if tr_post == 2:
#                 print(Date_column)
                # Quarter Column
                col_Q, col_year = Quarter( Date_column, k )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 2: 

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
#                 if len(cleaned_data) > 10:
#                     del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        # 10-K
        elif k == '10-K':
            if tr_post == 1 or tr_post ==2:
                
                Date_column.append(column)               
                
            elif tr_post == 3:
                
                # Quarter Column
                col_date, col_year = Quarter([item for sublist in Date_column for item in sublist], k)
                single_table.append( col_date )
                # Year Column
                single_table.append( col_year )
                
            elif tr_post > 3:              
                
                cleaned_data = cleaning_column( column , 10**6 )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                
                single_table.append( cleaned_data )
            
    return single_table

In [23]:
def df_clean( single_table ):
#     print(single_table)
    table_df = pd.DataFrame(single_table)

    table_df = table_df.transpose()  

    #table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [124]:
def table(page_code, k):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if anchor_check(page) and ('in millions' in page):
            
            single_table = table_extractor( page, 10**6, k)
            
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif anchor_check(page) and ('in thousands' in page):
            
            single_table = table_extractor( page, 10**3, k)
            
            table_df = df_clean(single_table)
            
            doc_tables.append(table_df)
            
                
    return doc_tables
    

# Special to deal with 10-Q

In [13]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [46]:
explore_dict = {'10-Q' : ['0001193125-13-303204',
  '0001193125-13-174667',
  '0001193125-12-436012']}

In [60]:
QorK_10Q_1 = {'10-Q': ['0001018724-20-000010',
  '0001018724-19-000089',
  '0001018724-19-000071',
  '0001018724-19-000043',
  '0001018724-18-000159',
  '0001018724-18-000108',
  '0001018724-18-000072',
  '0001018724-17-000135',
  '0001018724-17-000100',
  '0001018724-17-000051',
  '0001018724-16-000324',
  '0001018724-16-000286',
  '0001018724-16-000227',
  '0001018724-15-000126',
  '0001018724-15-000087',
  '0001018724-15-000038',
  '0001018724-14-000055',
  '0001018724-14-000025',
  '0001018724-14-000014',
  '0001445305-13-002495',
  '0001193125-13-303204']}

In [164]:
Checklist = {}

for k, v in QorK_10Q_1.items():
    value_list = []
    for i, vv in enumerate(v):
        year = int(vv.split(sep='-')[1])
        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

#         try:
        # Extract Table from html code and wrap it as a list of df.
        table_df = table(pages, k)[0]  
        
        if year > 16:
            table_df = table_df.drop([1]).reset_index(drop=True)
            
        else: 
            table_df = table_df.drop([2]).reset_index(drop=True)
        
        ICD.display(table_df)

        with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
            pickle.dump(table_df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
#         except: 
#             value_list.append(vv)
#             print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
#     del value_list

Q ['Q4', 'Q1']
Y - ['2019', '2020']


0 Quarter  Year   NaN ASSETS Current assets         Cash  \
0      Q1  2020  None   None           None  27201000000   

0 Marketable securities    Inventory Accounts Receivable Total current assets  \
0           22091000000  18857000000         17836000000          85985000000   

0 Property and Equipment Operating leases     Goodwill Other assets  \
0            77779000000      26279000000  14739000000  16456000000   

0  Total assets Shareholder Equity Current liabilities Accounts payable  \
0  221238000000               None                None      40056000000   

0 Accrued expenses and other Unearned revenue Total current liabilities  \
0                30791000000       8864000000               79711000000   

0 Long term lease liabilities Long term debt Other long term liabilities  \
0                 40300000000    23437000000                 12518000000   

0 Commitments and contingencies  Shareholder Equity Preferred Stock  \
0                           None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  521 and 522  \
0                 None                       None   

0 Outstanding shares  498 and 499 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                35412000000   

0 Accumulated other comprehensive income  Retained Earnings  \
0                              2063000000       33755000000   

0 Shareholder Equity Total liabilities and equity  
0        65272000000                 221238000000

All the pages from 10-Q document 0001018724-20-000010 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q4', 'Q3']
Y - ['2018', '2019']


0 Quarter  Year   NaN ASSETS Current assets         Cash  \
0      Q3  2019  None   None           None  23255000000   

0 Marketable securities    Inventory Accounts Receivable Total current assets  \
0           20146000000  18766000000         16887000000          79054000000   

0 Property and Equipment Operating leases     Goodwill Other assets  \
0            67662000000             None  14734000000  14535000000   

0  Total assets Shareholder Equity Current liabilities Accounts payable  \
0  199099000000               None                None      35794000000   

0 Accrued expenses and other Unearned revenue Total current liabilities  \
0                28961000000       7381000000               72136000000   

0 Long term lease liabilities Long term debt Other long term liabilities  \
0                 37058000000    22472000000                 10925000000   

0 Commitments and contingencies  Shareholder Equity Preferred Stock  \
0                           None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  514 and 519  \
0                 None                       None   

0 Outstanding shares  491 and 495 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                31817000000   

0 Accumulated other comprehensive loss Retained Earnings Shareholder Equity  \
0                           1429000000       27952000000        56508000000   

0 Total liabilities and equity  
0                 199099000000

All the pages from 10-Q document 0001018724-19-000089 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q4', 'Q2']
Y - ['2018', '2019']


...

All the pages from 10-Q document 0001018724-14-000014 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q3', 'Q4']
Y - ['2012', '2013']


0 Quarter  Year   NaN ASSETS Current assets        Cash Marketable securities  \
0      Q3  2013  None   None           None  3872000000            3817000000   

0   Inventory Accounts Receivable Deferred tax assets Total current assets  \
0  6068000000          3057000000           520000000          17334000000   

0 Property and Equipment Deferred tax assets    Goodwill Other assets  \
0             9991000000           128000000  2635000000   1773000000   

0 Total assets Shareholder Equity Current liabilities Accounts payable  \
0  31861000000               None                None      10037000000   

0 Accrued expenses and other Total current liabilities Long term debt  \
0                 6098000000               16135000000     3043000000   

0 Other long term liabilities Commitments and contingencies  \
0                  3596000000                          None   

0 Shareholder Equity Preferred Stock Authorized shares  500  \
0               None            None                   None   

0 Issued and outstanding shares  none Common Stock Authorized shares  5  \
0                                None         None                 None   

0 Issued shares  481 and 478 Outstanding shares  458 and 454 Treasury stock  \
0                       None                         5000000     1837000000   

0 Additional paid in capital Accumulated other comprehensive loss  \
0                 9175000000                            206000000   

0 Retained Earnings Shareholder Equity Total liabilities and equity  
0        1950000000         9087000000                  31861000000

All the pages from 10-Q document 0001445305-13-002495 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q2', 'Q4']
Y - ['2012', '2013']


0 Quarter  Year   NaN ASSETS Current assets        Cash Marketable securities  \
0      Q2  2013  None   None           None  3704000000            3759000000   

0   Inventory Accounts Receivable Deferred tax assets   NaN  \
0  5420000000          2861000000           541000000  None   

0 Total current assets Property and Equipment Deferred tax assets    Goodwill  \
0          16285000000             8789000000           128000000  2614000000   

0 Other assets   NaN Total assets   NaN Shareholder Equity  \
0   1807000000  None  29623000000  None               None   

0 Current liabilities Accounts payable Accrued expenses and other   NaN  \
0                None       8990000000                 5745000000  None   

0 Total current liabilities Long term debt Other long term liabilities  \
0               14735000000     3042000000                  3113000000   

0 Commitments and contingencies Shareholder Equity Preferred Stock  \
0                          None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  480 and 478  \
0                 None                       None   

0 Outstanding shares  457 and 454 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                 8893000000   

0 Accumulated other comprehensive loss Retained Earnings   NaN  \
0                            319000000        1991000000  None   

0 Shareholder Equity   NaN Total liabilities and equity   NaN  
0         8733000000  None                  29623000000  None

All the pages from 10-Q document 0001193125-13-303204 have been tableized and saved.
--------------------------------------------------------------------------------


In [165]:
Checklist

{'10-Q': []}

In [166]:
QorK_10Q_2 = {'10-Q' :['0001193125-13-174667',
  '0001193125-12-436012',
  '0001193125-12-317409',
  '0001193125-12-187203',
  '0001193125-11-280739',
  '0001193125-11-197823',
  '0001193125-11-110197',
  '0001193125-10-233883',
  '0001193125-10-164083',
  '0001193125-10-090367',
  '0001193125-09-212134',
  '0001193125-09-154174',
  '0001193125-09-085983',
  '0001193125-08-214483',
  '0001193125-08-157451',
  '0001193125-08-089548']}

In [165]:
Checklist

{'10-Q': []}

# Special 10-Q Ends

# ***************Making this only to rectify 10-Ks in Amazon***************

In [45]:
explore_10_K = {'10-K': ['0001018724-20-000004',
  '0001018724-19-000004',
  '0001018724-18-000005',
  '0001018724-17-000011',
  '0001018724-16-000172',
  '0001018724-15-000006',
  '0001018724-14-000006',
  '0001193125-13-028520',
  '0001193125-12-032846',
  '0001193125-11-016253',
  '0001193125-10-016098',
  '0001193125-09-014406',
  '0001193125-08-024707']}

In [71]:
for QK, N_O in explore_10_K.items():
    
    for an_postition, acc_num in enumerate(N_O): 
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df = table(pages, QK)
        
        # Hereinafter is executed in part 2.
        table_df = test_df[0].drop([1]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)


 This is 0001018724-20-000004 a 10-K document.
Q ['Q4']
this is the interger version of Y -  2019
Y - ['2019']
Shape -  (1, 37)


0 Quarter  Year Current assets         Cash Marketable securities  \
0      Q4  2019           None  36092000000           18929000000   

0    Inventory Accounts Receivable Total current assets  \
0  20497000000         20816000000          96334000000   

0 Property and Equipment Operating leases     Goodwill Other assets  \
0            72705000000             None  14754000000  16314000000   

0  Total assets Shareholder Equity Current liabilities Accounts payable  \
0  225248000000               None                None      47183000000   

0 Accrued expenses and other Unearned revenue Total current liabilities  \
0                32439000000       8190000000               87812000000   

0 Long term lease liabilities Long term debt Other long term liabilities  \
0                 39791000000    23414000000                 12171000000   

0 Commitments and contingencies  Shareholder Equity Preferred Stock  \
0                           None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  514 and 521  \
0                 None                       None   

0 Outstanding shares  491 and 498 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                33658000000   

0 Accumulated other comprehensive income  Retained Earnings  \
0                               986000000       31220000000   

0 Shareholder Equity Total liabilities and equity  
0        62060000000                 225248000000


 This is 0001018724-19-000004 a 10-K document.
Q ['Q4']
this is the interger version of Y -  2018
Y - ['2018']
Shape -  (1, 35)


0 Quarter  Year Current assets         Cash Marketable securities  \
0      Q4  2018           None  31750000000            9500000000   

0    Inventory Accounts Receivable Total current assets  \
0  17174000000         16677000000          75101000000   

0 Property and Equipment     Goodwill Other assets  Total assets  \
0            61797000000  14548000000  11202000000  162648000000   

0 Shareholder Equity Current liabilities Accounts payable  \
0               None                None      38192000000   

0 Accrued expenses and other Unearned revenue Total current liabilities  \
0                23663000000       6536000000               68391000000   

0 Long term debt Other long term liabilities Commitments and contingencies   \
0    23495000000                 27213000000                           None   

0 Shareholder Equity Preferred Stock Authorized shares  500  \
0               None            None                   None   

0 Issued and outstanding shares  none Common Stock Authorized shares  5  \
0                                None         None                 None   

0 Issued shares  507 and 514 Outstanding shares  484 and 491 Treasury stock  \
0                       None                         5000000     1837000000   

0 Additional paid in capital Accumulated other comprehensive loss  \
0                26791000000                           1035000000   

0 Retained Earnings Shareholder Equity Total liabilities and equity  
0       19625000000        43549000000                 162648000000


 This is 0001018724-18-000005 a 10-K document.
Q ['Q4']
this is the interger version of Y -  2017
Y - ['2017']
Shape -  (1, 35)


...

0 Quarter  Year Current assets        Cash Marketable securities   Inventory  \
0      Q4  2009           None  2769000000             958000000  1399000000   

0 Accounts Receivable Deferred tax assets   NaN Total current assets  \
0           827000000           204000000  None           6157000000   

0 Fixed assets Deferred tax assets   Goodwill Other assets   NaN Total assets  \
0    854000000           145000000  438000000    720000000  None   8314000000   

0   NaN L I A B I L I T I E SA N DS T O C K H O L D E R SE Q U I T Y  \
0  None                                               None             

0 Current liabilities Accounts payable Accrued expenses and other   NaN  \
0                None       3594000000                 1152000000  None   

0 Total current liabilities Long term debt Other long term liabilities  \
0                4746000000      409000000                   487000000   

0 Commitments and contingencies Shareholder Equity Preferred Stock  \
0                          None               None            None   

0 Authorized shares500 Issued and outstanding sharesnone Common Stock  \
0                 None                              None         None   

0 Authorized shares5 Issued shares461 and 445 Outstanding shares444 and 428  \
0               None                     None                       4000000   

0 Treasury stock Additional paid in capital  \
0      600000000                 4121000000   

0 Accumulated other comprehensive income  Retained Earnings   NaN  \
0                               123000000         730000000  None   

0 Shareholder Equity   NaN Total liabilities and equity   NaN  
0         2672000000  None                   8314000000  None


 This is 0001193125-09-014406 a 10-K document.
Q ['Q4']
this is the interger version of Y -  2007
Changed Y into ['2008']
Y - ['2008']
Shape -  (1, 44)


0 Quarter  Year Current assets        Cash Marketable securities   Inventory  \
0      Q4  2008           None  2539000000             573000000  1200000000   

0 Accounts Receivable Deferred tax assets   NaN Total current assets  \
0           705000000           147000000  None           5164000000   

0 Fixed assets Deferred tax assets   Goodwill Other assets   NaN Total assets  \
0    543000000           260000000  222000000    296000000  None   6485000000   

0   NaN L I A B I L I T I E SA N DS T O C K H O L D E R SE Q U I T Y  \
0  None                                               None             

0 Current liabilities Accounts payable Accrued expenses and other  \
0                None       2795000000                  902000000   

0 Current portion of long term debt   NaN Total current liabilities  \
0                          17000000  None                3714000000   

0 Long term debt Other long term liabilities Commitments and contingencies  \
0     1282000000                   292000000                          None   

0 Shareholder Equity Preferred Stock Authorized shares500  \
0               None            None                 None   

0 Issued and outstanding sharesnone Common Stock Authorized shares5  \
0                              None         None               None   

0 Issued shares445 and 431 Outstanding shares428 and 416 Treasury stock  \
0                     None                       4000000      500000000   

0 Additional paid in capital Accumulated other comprehensive income   \
0                 3063000000                                 5000000   

0 Accumulated deficit   NaN Shareholder Equity   NaN  \
0          1375000000  None         1197000000  None   

0 Total liabilities and equity   NaN  
0                   6485000000  None


 This is 0001193125-08-024707 a 10-K document.
Q ['Q4']
this is the interger version of Y -  2006
Changed Y into ['2007']
Y - ['2007']
Shape -  (1, 43)


0 Quarter  Year Current assets        Cash Marketable securities  Inventory  \
0      Q4  2007           None  1022000000             997000000  877000000   

0 Accounts Receivable Deferred tax assets   NaN Total current assets  \
0           399000000            78000000  None           3373000000   

0 Fixed assets Deferred tax assets   Goodwill Other assets   NaN Total assets  \
0    457000000           199000000  195000000    139000000  None   4363000000   

0   NaN L I A B I L I T I E SA N DS T O C K H O L D E R SE Q U I T Y  \
0  None                                               None             

0 Current liabilities Accounts payable Accrued expenses and other   NaN  \
0                None       1816000000                  716000000  None   

0 Total current liabilities Long term debt Other long term liabilities  \
0                2532000000     1247000000                   153000000   

0 Commitments and contingencies Shareholder Equity Preferred Stock  \
0                          None               None            None   

0 Authorized shares500 Issued and outstanding sharesnone Common Stock  \
0                 None                              None         None   

0 Authorized shares5 Issued shares431 and 422 Outstanding shares416 and 414  \
0               None                     None                       4000000   

0 Treasury stock Additional paid in capital  \
0      252000000                 2517000000   

0 Accumulated other comprehensive income  Accumulated deficit   NaN  \
0                                 1000000          1837000000  None   

0 Shareholder Equity   NaN Total liabilities and equity   NaN  
0          431000000  None                   4363000000  None

# ***************Special Save for 10-Ks***************


In [73]:
Checklist = {}

for k, v in explore_10_K.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            table_df = table(pages, k)                 
            #ICD.display(table_df[0])

            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list

Q ['Q4']
this is the interger version of Y -  2019
Y - ['2019']
All the pages from 10-K document 0001018724-20-000004 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q4']
this is the interger version of Y -  2018
Y - ['2018']
All the pages from 10-K document 0001018724-19-000004 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q4']
this is the interger version of Y -  2017
Y - ['2017']
All the pages from 10-K document 0001018724-18-000005 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q4']
this is the interger version of Y -  2016
Changed Y into ['2017']
Y - ['2017']
All the pages from 10-K document 0001018724-17-000011 have been tableized and saved.
--------------------------------------------------------------------------------
Q ['Q4']
this is the interger version of Y -  2014
Chang

# ***************10-K special ends***************

# Exploring the Data

In [21]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [22]:
explore_dict

{'10-Q': ['0001018724-20-000010', '0001193125-08-089548'],
 '10-K': ['0001018724-20-000004', '0001193125-08-024707']}

In [40]:
for QK, N_O in explore_dict.items():
    
    for an_postition, acc_num in enumerate(N_O): 
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df = table(pages, QK)
        
        table_df = test_df[0].drop([1]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)


 This is 0001018724-20-000010 a 10-Q document.
Q ['Q4', 'Q1']
Y - [[], [], [['2019']], [], [], [['2020']]]
Shape -  (1, 39)


0 Quarter  Year   NaN ASSETS Current assets         Cash  \
0      Q1  2020  None   None           None  27201000000   

0 Marketable securities    Inventory Accounts Receivable Total current assets  \
0           22091000000  18857000000         17836000000          85985000000   

0 Property and Equipment Operating leases     Goodwill Other assets  \
0            77779000000      26279000000  14739000000  16456000000   

0  Total assets Shareholder Equity Current liabilities Accounts payable  \
0  221238000000               None                None      40056000000   

0 Accrued expenses and other Unearned revenue Total current liabilities  \
0                30791000000       8864000000               79711000000   

0 Long term lease liabilities Long term debt Other long term liabilities  \
0                 40300000000    23437000000                 12518000000   

0 Commitments and contingencies  Shareholder Equity Preferred Stock  \
0                           None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  521 and 522  \
0                 None                       None   

0 Outstanding shares  498 and 499 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                35412000000   

0 Accumulated other comprehensive income  Retained Earnings  \
0                              2063000000       33755000000   

0 Shareholder Equity Total liabilities and equity  
0        65272000000                 221238000000


 This is 0001193125-08-089548 a 10-Q document.
Q ['Q1', 'Q4']
Y - [[], [['2008']], [], [['2007']]]
Shape -  (1, 46)


0 Quarter  Year   NaN ASSETS Current assets        Cash Marketable securities  \
0      Q4  2007  None   None           None  2539000000             573000000   

0   Inventory Accounts Receivable Deferred tax assets   NaN  \
0  1200000000           705000000           147000000  None   

0 Total current assets Fixed assets Deferred tax assets   Goodwill  \
0           5164000000    543000000           260000000  222000000   

0 Other assets   NaN Total assets   NaN Shareholder Equity  \
0    296000000  None   6485000000  None               None   

0 Current liabilities Accounts payable Accrued expenses and other  \
0                None       2795000000                  902000000   

0 Current portion of long term debt   NaN Total current liabilities  \
0                          17000000  None                3714000000   

0 Long term debt Other long term liabilities Commitments and contingencies  \
0     1282000000                   292000000                          None   

0 Shareholder Equity Preferred Stock Authorized shares  500  \
0               None            None                   None   

0 Issued and outstanding shares  none Common Stock Authorized shares  5  \
0                                None         None                 None   

0 Issued shares  432 and 431 Outstanding shares  417 and 416 Treasury stock  \
0                       None                         4000000      500000000   

0 Additional paid in capital Accumulated other comprehensive income  \
0                 3063000000                                5000000   

0 Accumulated deficit   NaN Shareholder Equity   NaN  \
0          1375000000  None         1197000000  None   

0 Total liabilities and equity   NaN  
0                   6485000000  None


 This is 0001018724-20-000004 a 10-K document.
Q ['Q4']
Y - [[], [], [['2018']], [['2019']]]
Shape -  (1, 37)


0 Quarter  Year Current assets         Cash Marketable securities  \
0    None  2019           None  36092000000           18929000000   

0    Inventory Accounts Receivable Total current assets  \
0  20497000000         20816000000          96334000000   

0 Property and Equipment Operating leases     Goodwill Other assets  \
0            72705000000             None  14754000000  16314000000   

0  Total assets Shareholder Equity Current liabilities Accounts payable  \
0  225248000000               None                None      47183000000   

0 Accrued expenses and other Unearned revenue Total current liabilities  \
0                32439000000       8190000000               87812000000   

0 Long term lease liabilities Long term debt Other long term liabilities  \
0                 39791000000    23414000000                 12171000000   

0 Commitments and contingencies  Shareholder Equity Preferred Stock  \
0                           None               None            None   

0 Authorized shares  500 Issued and outstanding shares  none Common Stock  \
0                   None                                None         None   

0 Authorized shares  5 Issued shares  514 and 521  \
0                 None                       None   

0 Outstanding shares  491 and 498 Treasury stock Additional paid in capital  \
0                         5000000     1837000000                33658000000   

0 Accumulated other comprehensive income  Retained Earnings  \
0                               986000000       31220000000   

0 Shareholder Equity Total liabilities and equity  
0        62060000000                 225248000000


 This is 0001193125-08-024707 a 10-K document.
Q ['Q4']
Y - [[], [], [['2007']], [['2006']]]
Shape -  (1, 43)


0 Quarter  Year Current assets        Cash Marketable securities  Inventory  \
0    None  2006           None  1022000000             997000000  877000000   

0 Accounts Receivable Deferred tax assets   NaN Total current assets  \
0           399000000            78000000  None           3373000000   

0 Fixed assets Deferred tax assets   Goodwill Other assets   NaN Total assets  \
0    457000000           199000000  195000000    139000000  None   4363000000   

0   NaN L I A B I L I T I E SA N DS T O C K H O L D E R SE Q U I T Y  \
0  None                                               None             

0 Current liabilities Accounts payable Accrued expenses and other   NaN  \
0                None       1816000000                  716000000  None   

0 Total current liabilities Long term debt Other long term liabilities  \
0                2532000000     1247000000                   153000000   

0 Commitments and contingencies Shareholder Equity Preferred Stock  \
0                          None               None            None   

0 Authorized shares500 Issued and outstanding sharesnone Common Stock  \
0                 None                              None         None   

0 Authorized shares5 Issued shares431 and 422 Outstanding shares416 and 414  \
0               None                     None                       4000000   

0 Treasury stock Additional paid in capital  \
0      252000000                 2517000000   

0 Accumulated other comprehensive income  Accumulated deficit   NaN  \
0                                 1000000          1837000000  None   

0 Shareholder Equity   NaN Total liabilities and equity   NaN  
0          431000000  None                   4363000000  None

# Save Files Locally. 

In [ ]:
Checklist = {}

for k, v in QorK.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            table_df = table(pages, k)                 
            #ICD.display(table_df[0])

            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list

In [ ]:
# Checklist of erroneous filings.  
print('These have a problem.')
Checklist

In [42]:
# Bring QorK over to Part 2. 
QorK

{'10-Q': ['0001018724-20-000010',
  '0001018724-19-000089',
  '0001018724-19-000071',
  '0001018724-19-000043',
  '0001018724-18-000159',
  '0001018724-18-000108',
  '0001018724-18-000072',
  '0001018724-17-000135',
  '0001018724-17-000100',
  '0001018724-17-000051',
  '0001018724-16-000324',
  '0001018724-16-000286',
  '0001018724-16-000227',
  '0001018724-15-000126',
  '0001018724-15-000087',
  '0001018724-15-000038',
  '0001018724-14-000055',
  '0001018724-14-000025',
  '0001018724-14-000014',
  '0001445305-13-002495',
  '0001193125-13-303204',
  '0001193125-13-174667',
  '0001193125-12-436012',
  '0001193125-12-317409',
  '0001193125-12-187203',
  '0001193125-11-280739',
  '0001193125-11-197823',
  '0001193125-11-110197',
  '0001193125-10-233883',
  '0001193125-10-164083',
  '0001193125-10-090367',
  '0001193125-09-212134',
  '0001193125-09-154174',
  '0001193125-09-085983',
  '0001193125-08-214483',
  '0001193125-08-157451',
  '0001193125-08-089548'],
 '10-K': ['0001018724-20-0000